In [1]:
from sklearn.datasets import load_iris
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [2]:
np.random.seed(1)

In [3]:
data = load_iris()

In [4]:
data['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [5]:
data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [6]:
data['target_names']

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

# prepare to create model

In [7]:
input_dim = 4
hidden_dim = 5
output_dim = 3

In [8]:
def printtensor(inputx,feedx,feedy):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        out = print(sess.run(inputx,feed_dict={X:feedx,y:feedy}))
        sess.close()
    return out   

In [9]:
X  = tf.placeholder(shape=(None,input_dim),dtype=tf.float64)
y  = tf.placeholder(shape=(None,output_dim),dtype=tf.float64)

In [10]:
train_x,test_x,train_y,test_y = train_test_split(data.data, data.target, test_size=0.33, random_state=40)

train_y = train_y.reshape(-1,1)
test_y = test_y.reshape(-1,1)

In [11]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
print(y.shape)

(100, 4)
(100,)
(50, 4)
(50,)
(?, 3)


In [12]:
one_hot_train = tf.one_hot(train_y,3,dtype=tf.float64)
one_hot_test = tf.one_hot(test_y,3,dtype=tf.float64)

In [13]:
ih_wts = tf.Variable(tf.random_normal([input_dim,hidden_dim], dtype=tf.float64))
h_biases = tf.Variable(1.0, dtype=tf.float64)
ho_wts = tf.Variable(tf.random_normal([hidden_dim,output_dim], dtype=tf.float64))
o_biases = tf.Variable(1.0, dtype=tf.float64)

#  define model


In [14]:
hidden = tf.nn.sigmoid(tf.add(tf.matmul(X,ih_wts),h_biases))
output = tf.nn.sigmoid(tf.add(tf.matmul(hidden,ho_wts),o_biases))

In [15]:
#hidden = tf.layers.dense(a, 5, tf.nn.relu, name="hidden")
#output = tf.layers.dense(hidden, 3, tf.nn.relu, name="output")

In [16]:
y_predict = tf.argmax(output,axis=1)

# Set up training

In [17]:
learn_rate = 0.001
max_epochs = 200

In [18]:
cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=one_hot_train, logits=output))

In [19]:
optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(cost)

# train model

In [20]:
init = tf.global_variables_initializer()

In [21]:
with tf.Session() as sess:
            sess.run(init)
            print("Starting training")
            print("====================================")
            for epoch in range(max_epochs):
                indices = np.arange(len(train_x))
                #convering tensor into array
                train_y_array=one_hot_train.eval()
                test_y_array=one_hot_test.eval()
                y_predict_float = tf.cast(output,dtype=tf.float64)
                #print(y_predict_float.shape)
                for ii in range(len(indices)):
                    i = indices[ii]
                    sess.run(optimizer,feed_dict={X:train_x[i:i+1],y:train_y_array[i:i+1]})
                    #printtensor(optimizer,train_x,train_y_array)
                #train_acc = np.mean(train_y_array-y_predict)
                train_calc = np.argmax(train_y_array, axis=1) == sess.run(y_predict_float, feed_dict={X: train_x, y: train_y_array})
                #print(train_calc)
                train_acc = np.mean(-train_y_array + sess.run(y_predict_float, feed_dict={X: train_x, y: train_y_array}))                
                if epoch > 0 and epoch % 10 == 0:
                    print("epoch = %4d, train accuracy = %.4f " % (epoch, train_acc))
            print("====================================")
            print("Training complete \n")
            print("====================================")
            print("Cost for training data:",sess.run(cost, feed_dict={X:train_x, y:train_y_array}))
            print("====================================")
            test_acc = np.mean(-test_y_array + sess.run(y_predict_float, feed_dict={X:test_x, y:test_y_array}))
            print("Accuracy on test data = %.4f " % test_acc)
            sess.close()
            print("====================================")
            print("\nEnd demo")
            print("====================================")


Starting training


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


epoch =   10, train accuracy = 0.4064 
epoch =   20, train accuracy = 0.4063 
epoch =   30, train accuracy = 0.4063 
epoch =   40, train accuracy = 0.4062 
epoch =   50, train accuracy = 0.4062 
epoch =   60, train accuracy = 0.4061 
epoch =   70, train accuracy = 0.4060 
epoch =   80, train accuracy = 0.4060 
epoch =   90, train accuracy = 0.4059 
epoch =  100, train accuracy = 0.4059 
epoch =  110, train accuracy = 0.4058 
epoch =  120, train accuracy = 0.4057 
epoch =  130, train accuracy = 0.4057 
epoch =  140, train accuracy = 0.4056 
epoch =  150, train accuracy = 0.4056 
epoch =  160, train accuracy = 0.4055 
epoch =  170, train accuracy = 0.4054 
epoch =  180, train accuracy = 0.4054 
epoch =  190, train accuracy = 0.4053 
Training complete 

Cost for training data: 1.0966563901859967
Accuracy on test data = 0.4052 

End demo
